In [ ]:
!sudo apt-get install libkrb5-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libkrb5-dev is already the newest version (1.17-6ubuntu4.3).
libkrb5-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:
pip install rasterio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 45.7 MB/s eta 0:00:00


In [ ]:
pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import math
import rasterio

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
with rasterio.open('srtm_37_03.tif') as ds:
  raw_arr = ds.read()
rows = raw_arr.shape[1]
columns = raw_arr.shape[2]
print(raw_arr.shape)
print(rows)
print(columns)


In [ ]:
arr=rasterio.open('srtm_37_03.tif')
xmin=arr.bounds[0]
ymin=arr.bounds[1]
xmax=arr.bounds[2]
ymax=arr.bounds[3]
cell_height=arr.height
cell_width=arr.width

In [ ]:
def getCoordinates(n,m):
    coordinates = []
    # +0.5  to get the longitude and latitude from the center of each cell
    lon = xmin + (m+0.5) * cell_width
    lat = ymin + ((rows - 1 - n +0.5) * cell_height)
    coordinates.append(lon)
    coordinates.append(lat)
    #print('coordinates is', coordinates)
    return coordinates

In [ ]:
def getDistance1(Lat_A,Lng_A,Lat_B,Lng_B): #Haversine with 2 radius
    ra=6378.140 #Equatorial radius
    rb=6356.755 #Polar radius
    flatten=(ra-rb)/ra  # calculating oblateness
    rad_lat_A=math.radians(Lat_A)
    rad_lng_A=math.radians(Lng_A)
    rad_lat_B=math.radians(Lat_B)
    rad_lng_B=math.radians(Lng_B)
    pA=math.atan(rb/ra*math.tan(rad_lat_A))
    pB=math.atan(rb/ra*math.tan(rad_lat_B))
    xx=math.acos(math.sin(pA)*math.sin(pB)+math.cos(pA)*math.cos(pB)*math.cos(rad_lng_A-rad_lng_B))
    c1=(math.sin(xx)-xx)*(math.sin(pA)+math.sin(pB))**2/math.cos(xx/2)**2
    c2=(math.sin(xx)+xx)*(math.sin(pA)-math.sin(pB))**2/math.sin(xx/2)**2
    dr=flatten/8*(c1-c2)
    distance=ra*(xx+dr)
    return distance

In [ ]:
def getDistance2(lat1,lng1,lat2,lng2):# Haversine distance which is calculated with averaged radius between two points
    radlat1=math.radians(lat1)
    radlat2=math.radians(lat2)
    a=radlat1-radlat2
    b=math.radians(lng1)-math.radians(lng2)
    s=2*math.asin(math.sqrt(pow(math.sin(a/2),2)+math.cos(radlat1)*math.cos(radlat2)*pow(math.sin(b/2),2)))
    #earth_radius=6378.137
    earth_radius=6371.009 # averaged radius
    s=s*earth_radius
    if s<0:
        return -s
    else:
        return s

In [ ]:
def getGradient(n_a, m_a, n_b, m_b):
    coord_lat_a = getCoordinates(n_a, m_a)[1]
    coord_lon_a = getCoordinates(n_a, m_a)[0]
    coord_lat_b = getCoordinates(n_b, m_b)[1]
    coord_lon_b = getCoordinates(n_b, m_b)[0]
    height = raw_arr[n_b][m_b] - raw_arr[n_a][m_a]
    # *1000 because getdistance function returns the km
    distance_ab = getDistance1(coord_lat_a,coord_lon_a, coord_lat_b,coord_lon_b)*1000
    gradient = math.degrees(math.atan((height/distance_ab)))
    g1=np.vectorize(gradient)
    return g1


In [ ]:
def dijkstra_gradient_threshold(startNode_n, startNode_m, endNode_n, endNode_m, gradient_threshold):
    # to match with the all array start from 0
    startNode_n = startNode_n-1
    startNode_m = startNode_m-1
    endNode_n = endNode_n-1
    endNode_m = endNode_m-1

    rowArr_shape = arr.shape
    # This array shows the node is visited or not, the initial array
    # is all 0 which means all nodes are unvisited so far, once visited will turn to 1
    visitUnvisitArr = np.zeros(rowArr_shape, dtype = int)
    #if all the nodes are visited in the end the arr should be the same as this one (used for comparision and end the lope)
    #allvisited_arr = np.full(rowArr_shape, 1,dtype = np.int)

    # this array stores the shortest distance. the initial value is inf which should means it is infinite

    shortDistArr = np.full(rowArr_shape,np.inf)

    #shortDistArr = np.zeros(rowArr_shape, dtype = np.int)
    # preVertex array stores all the preVertex this is node (for calculate the whole shortest path in the end)
    preVertex = [[[] for col in range(raw_arr.shape[1])] for row in range(raw_arr.shape[0])]
    # give the previous node of the start node it self for preventing later problem
    preVertex[startNode_n][startNode_m].append(startNode_n)
    preVertex[startNode_n][startNode_m].append(startNode_m)

    # the distance to itself is 0.
    shortDistArr[startNode_n][startNode_m] = 0
    # set the start node as current node for starting loop
    current_node_n = startNode_n
    current_node_m = startNode_m
    # loop until the all the nodes are visited
    #while not ((visitUnvisitArr == allvisited_arr).all()):

    # loop until the end node is visited or break if there are no nodes available due to current parameter
    while visitUnvisitArr[endNode_n][endNode_m] != 1:
        # update shortest distance of 9 cells around the current node
        for i in range(current_node_m-1,current_node_m+2):
            # prevent out of bound of array
            if i < 0 or i > (raw_arr.shape[1]-1):
                continue
            for j in range(current_node_n-1,current_node_n+2):

                if j < 0 or j > (raw_arr.shape[0]-1):
                    continue
                # skip itself
                elif i == current_node_m and j == current_node_n:
                    continue
                # skip the visited node
                elif visitUnvisitArr[j][i] == 1:
                    continue
                # If the gradient is too high then this node can not be reached
                elif abs(getGradient(current_node_n,current_node_m,j,i)) > gradient_threshold:
                    #print("No way ahead",getGradient(current_node_n,current_node_m,j,i))
                    continue
                else:
                    # these are the 8 connected nodes around this cell   ( np.inf == np.inf +1 )
                    # update the shortest distance. "Relaxation step"!!!
                    if shortDistArr[j][i] > (shortDistArr[current_node_n][current_node_m] + getDistance1(getCoordinates(current_node_n,current_node_m)[1],
                                                                                                         getCoordinates(current_node_n,current_node_m)[0],
                                                                                                         getCoordinates(j,i)[1],getCoordinates(j,i)[0])):

                        shortDistArr[j][i] = (shortDistArr[current_node_n][current_node_m] + getDistance1(getCoordinates(current_node_n,current_node_m)[1],
                                                                                                          getCoordinates(current_node_n,current_node_m)[0],
                                                                                                          getCoordinates(j,i)[1],getCoordinates(j,i)[0]))

                        # put previous vertex [j,i] in to cell for later calculate the whole path
                        # some nodes are "relaxated" before so there are values already there. do not append use update!
                        if preVertex[j][i] ==[]:
                            preVertex[j][i].append(current_node_n)
                            preVertex[j][i].append(current_node_m)
                        elif preVertex[j][i] !=[]:
                            preVertex[j][i][0] = current_node_n
                            preVertex[j][i][1] = current_node_m

        # 1 means already visited and will not be visit again
        visitUnvisitArr[current_node_n][current_node_m] = 1

        # once reached the end node this job is done
        if visitUnvisitArr[endNode_n][endNode_m] == 1:
            continue

        temp_low = np.inf
        temp_low_n = None
        temp_low_m = None

        # find the lowest value and the cell in the all unvisited nodes
        for q in range(shortDistArr.shape[0]):
            for w in range(shortDistArr.shape[1]):
                if visitUnvisitArr[q][w] == 1:
                    continue
                elif visitUnvisitArr[q][w] ==0:
                    if temp_low > shortDistArr[q][w]:
                        temp_low = shortDistArr[q][w]
                        temp_low_n = q
                        temp_low_m = w
        if temp_low_n is None and temp_low_m is None:
            current_node_n = temp_low_n
            current_node_m = temp_low_m
        else:
            the_shortest_distance = shortDistArr[endNode_n][endNode_m]
            # store the shortest path
            the_shortest_path = []
            pre_node_n = preVertex[endNode_n][endNode_m][0]
            pre_node_m = preVertex[endNode_n][endNode_m][1]
            the_shortest_path.insert(0,[endNode_n,endNode_m])

            # iterate to the start node

            while pre_node_n != startNode_n or pre_node_m != startNode_m:
                the_shortest_path.insert(0,[pre_node_n,pre_node_m])
                pre_node_n = preVertex[pre_node_n][pre_node_m][0]
                pre_node_m = preVertex[pre_node_n][pre_node_m][1]

            else:
                the_shortest_path.insert(0, [startNode_n, startNode_m])
                print('the shortest distance :',the_shortest_distance)
                print('Path :',the_shortest_path)

                # this array is only for visualizing the path.
                demonstrate_array = np.full(rowArr_shape,np.nan)
                for p in range(len(the_shortest_path)):
                    demonstrate_array[the_shortest_path[p][0]][the_shortest_path[p][1]] = 0
                print("Route:",demonstrate_array)

                # in this path the coordinates are geographic coordinates
                geo_path = []
                # turn the numpy coordinate back to geographical coordinates
                for np_coord in the_shortest_path:
                    geo_node = []
                    geo_node.append(getCoordinates(np_coord[0], np_coord[1])[0])
                    geo_node.append(getCoordinates(np_coord[0], np_coord[1])[1])
                    geo_path.append(geo_node)
                print('geo_path',geo_path)

                # create the path (polyline) shape file
                # A list that will hold each of the Polyline objects/ in this case only one polyline
                path = []
                path.append(geo_path)


In [ ]:
from ipywidgets import widgets
lbl1=widgets.Label('Enter the start point row number')
display(lbl1)
text1=widgets.Text()
display(text1)
lbl2=widgets.Label('Enter the start point column number')
display(lbl2)
text2=widgets.Text()
display(text2)
lbl3=widgets.Label('Enter the end point row number')
display(lbl3)
text3=widgets.Text()
display(text3)
lbl4=widgets.Label('Enter the end point column number')
display(lbl4)
text4=widgets.Text()
display(text4)
lbl5=widgets.Label('Enter the gradient threshold')
display(lbl5)
text5=widgets.Text()
display(text5)
btn=widgets.Button(description='enter')
display(btn)
def smooth(b):
  input_start_n=int(text1.value)
  input_start_m=int(text2.value)
  input_end_n=int(text3.value)
  input_end_m=int(text4.value)
  input_gradient_threshold=float(text5.value)
  dijkstra_gradient_threshold(input_start_n, input_start_m, input_end_n, input_end_m, input_gradient_threshold)
btn.on_click(smooth)








Label(value='Enter the start point row number')

Text(value='')

Label(value='Enter the start point column number')

Text(value='')

Label(value='Enter the end point row number')

...

Text(value='')

Label(value='Enter the gradient threshold')

Text(value='')

Button(description='enter', style=ButtonStyle())

NameError: ignored